In [ ]:
def get_layer_feature(model, feature_layer_name, image):
    # image = self.normalize(self.toTensor(img)).unsqueeze(0).to(self.device)
    # embedding = torch.zeros(image.shape[0], num_features, 1, 1).to(image.device)
    feature_layer = model._modules.get(feature_layer_name)

    embedding = []

    def copyData(module, input, output):
        embedding.append(output.data)

    h = feature_layer.register_forward_hook(copyData)
    out = model(image.to(image.device))
    h.remove()
    embedding = embedding[0]
    assert embedding.shape[0] == image.shape[0], f"{embedding.shape[0]} != {image.shape[0]}"
    assert embedding.shape[2] == 1, f"{embedding.shape[2]} != 1"
    assert embedding.shape[2] == 1, f"{embedding.shape[3]} != 1"
    return embedding[:, :, 0, 0]

def batch_func(batch):
    with torch.no_grad():
        efficientnet_feature = get_layer_feature(
            efficientnet, "avgpool", batch["image"].to(efficientnet_device)
        )

    return {
        "efficientnet_feature": efficientnet_feature.detach().cpu(),
        "metadata": batch["metadata"],
    }

efficientnet_device="cuda:6"
efficientnet = torchvision.models.efficientnet_v2_s(
    weights=torchvision.models.EfficientNet_V2_S_Weights.IMAGENET1K_V1
).to(efficientnet_device)
efficientnet.eval()

print("Featurizing...")
loader_applied = dataloader_apply_func(
    dataloader=test_dataloader_f17k,
    func=batch_func,
    collate_fn=custom_collate_per_key,
)
image_features_f17k_efficientnet=loader_applied["efficientnet_feature"]

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50, svd_solver="auto")
print("Fitting PCA...")
pca.fit(image_features_f17k_efficientnet)
image_features_f17k_efficientnet_pc=pca.transform(image_features_f17k_efficientnet)
# if feature.shape[0] > 100000:
#     pca.fit(
#         feature[
#             np.random.RandomState(42).choice(
#                 np.arange(len(feature)),
#                 size=10000,
#                 replace=False,
#             )
#         ]
#     )
# else:

In [ ]:
import sklearn.metrics
image_features_f17k_efficientnet_pc_cos=sklearn.metrics.pairwise.cosine_similarity(image_features_f17k_efficientnet_pc)
image_features_f17k_efficientnet_pc_cos.shape

In [ ]:
cutoff=0.1
features=1-image_features_f17k_efficientnet_pc_cos
group_idx=pd.Series([np.nan]*len(test_dataloader_f17k.dataset.metadata_all.index), index=test_dataloader_f17k.dataset.metadata_all.index)
# dup_set_list=[]


for idx_count, (idx, row) in enumerate(test_dataloader_f17k.dataset.metadata_all.iterrows()):
#     diff_array_sort=np.sort(1-image_features_f17k_efficientnet_pc_cos[idx_count])
#     diff_array_argsort=np.argsort(1-image_features_f17k_efficientnet_pc_cos[idx_count])
#     print(features[])
    if (features[idx_count]<cutoff).sum()==1:
        continue
        
#     print(idx, idx_count)
#     print((features[idx_count]<cutoff))
    group_idx_select=group_idx[features[idx_count]<cutoff]
    group_idx_select_nonull=group_idx_select[~group_idx_select.isnull()]
#     print()

    if idx=="32acc72c4d4c0d7659280c2dba57b948.jpg":
        print(group_idx[features[idx_count]<cutoff], len(group_idx_select_nonull), idx)

    if len(group_idx_select_nonull)==0:
        group_idx[features[idx_count]<cutoff]=idx
    else:
        group_idx[features[idx_count]<cutoff]=group_idx_select_nonull.iloc[0]
    if idx=="32acc72c4d4c0d7659280c2dba57b948.jpg":
        print(group_idx[features[idx_count]<cutoff], len(group_idx_select_nonull), idx)        
    
#     print(group_idx_select[~group_idx_select.isnull()].iloc[0])
    print((features[idx_count]<cutoff).sum(), '    ', idx_count)
    
#     break
#     for diff_idx, diff in zip(diff_array_argsort, diff_array_sort):
#         if diff>=cutoff:
#             break        
#         for dup_set in dup_set_list:
#             if diff_idx in dup_set:
#                 dup_set.add()
        

In [ ]:
start_idx=4500
end_idx=4500+500

n_top=5
n_row=((((1-image_features_f17k_efficientnet_pc_cos[start_idx:end_idx])<0.1).sum(axis=1)-1)>0).sum()
cutoff=0.1


fig, axes = plt.subplots(nrows=n_row, ncols=n_top, figsize=(5,n_row))

row_count=-1
for idx_count, (idx, row) in enumerate(test_dataloader_f17k.dataset.metadata_all.iterrows()):
    if row_count+1==n_row:
        break
        
    if idx_count not in range(start_idx,end_idx):
        continue
        
        
    diff_array_sort=np.sort(1-image_features_f17k_efficientnet_pc_cos[idx_count])
    diff_array_argsort=np.argsort(1-image_features_f17k_efficientnet_pc_cos[idx_count])
    
    
    if diff_array_sort[1]>=cutoff:
        continue
    
    col_count=0
    row_count+=1
    plotted_idx=[]
    for diff_idx, diff in zip(diff_array_argsort, diff_array_sort):
        if diff>=cutoff:
            break                        
        if col_count>=n_top:
            break
#         if diff_idx<idx_count:
#             continue
        plotted_idx.append(diff_idx)
        axes[row_count, col_count].imshow(Image.open(test_dataloader_f17k.dataset.image_path_dict[
                                            test_dataloader_f17k.dataset.metadata_all.index[diff_idx]
                                        ]).resize((200,200)))
        
        axes[row_count, col_count].set_xticks([])
        axes[row_count, col_count].set_yticks([])         
        
        axes[row_count, col_count].set_title(str(diff_idx)+', '+f"{diff:.2f}", y=0.6, fontdict={'color': 'red', 
                                                                                               "fontsize":7})
        col_count+=1
    print(str(plotted_idx)+",")
        
        
    for col_count_idx in range(col_count, n_top):
        axes[row_count, col_count_idx].set_xticks([])
        axes[row_count, col_count_idx].set_yticks([])                 

In [ ]:
dup_set_list_new=[]
for i in range(len(dup_set_list)):
    merged=False
    for element in dup_set_list[i]:
        if merged:
            break
        for j in range(len(dup_set_list_new)):
            if element in dup_set_list_new[j]:
                dup_set_list_new[j]=dup_set_list_new[j]+dup_set_list[i]
                merged=True
                break
    if not merged:
        dup_set_list_new.append(dup_set_list[i])
dup_set_list_new=[list(set(dup_set)) for dup_set in dup_set_list_new]        

In [ ]:
dup_set_list_new_hash=[[test_dataloader_f17k.dataset.metadata_all.index[dup_idx] for dup_idx in dup_set] for dup_set in dup_set_list_new]

In [ ]:
n_top=6
n_row=523
dup_set_list_new_hash
dup_set_list_new_hash_sorted=[]

fig, axes = plt.subplots(nrows=n_row, ncols=n_top, figsize=(5,n_row))

row_count=-1
for idx_count, dup_set in enumerate(dup_set_list_new_hash):
    if row_count+1==n_row:
        break
        
    dup_set=test_dataloader_f17k.dataset.metadata_all.loc[dup_set]["skincon_Nodule"].fillna(-9).sort_values(ascending=False).index.tolist()
    dup_set_list_new_hash_sorted.append(dup_set)
    
    col_count=0
    row_count+=1
    plotted_idx=[]
    for dup_idx in dup_set:
        if col_count>=n_top:
            break

        plotted_idx.append(dup_idx)
        axes[row_count, col_count].imshow(Image.open(test_dataloader_f17k.dataset.image_path_dict[
                                            dup_idx
                                        ]).resize((200,200)))
        
        axes[row_count, col_count].set_xticks([])
        axes[row_count, col_count].set_yticks([])         
        
        if np.isnan(test_dataloader_f17k.dataset.metadata_all.loc[dup_idx]["skincon_Nodule"])==False:
            axes[row_count, col_count].set_title("skincon", y=0.6, fontdict={'color': 'red', 
                                                                                               "fontsize":7})
        col_count+=1
    print(str(plotted_idx)+",")
        
        
    for col_count_idx in range(col_count, n_top):
        axes[row_count, col_count_idx].set_xticks([])
        axes[row_count, col_count_idx].set_yticks([])                 

In [ ]:
print(pd.DataFrame(dup_set_list_new_hash_sorted).to_csv())

In [ ]:
# 0 304
# 1 195
# 2 24